Make the import

In [ ]:
import os
import logging
import sys
import numpy as np
import mne
import myonset as myo

Set the path

In [ ]:
# where bdf data is
#path_bdf = os.path.join('.','bdf')
path_bdf = os.path.join('.')

# where automatic markers file is 
#path_mrk = os.path.join('.','automatic_detection')
path_mrk = os.path.join('.')

# where you want to save the corrected markers
#path_corrected_mrk = os.path.join('.','corrected_detection')
path_corrected_mrk = os.path.join('.')


Set trigger id values 

In [ ]:
#triggers values used for trial segmentation (e.g., stimulus or fixation cross)
stim_id = {'red_left':    12,\
           'red_right':   22,\
           'green_left':  11,\
           'green_right': 21}

#triggers values used for detected onset, offset and peak events
emg_id = {'onset': 131, 'offset': 132, 'peak': 133}

# list of response triggers, used to remove bursts detected after response 
resp_list =  [100,200] 

# Load the BDF file

Set EMG file name

In [ ]:
name_bdf = 's1.bdf'

Extract file name and create log file

In [ ]:
nameSubj = name_bdf.split('.')[0]
fname = os.path.join(path_bdf,name_bdf)

logName = os.path.join(path_bdf,nameSubj + '.log')
logging.basicConfig(filename=logName, level=logging.INFO) # needed in jupyter notebook
myo.set_log_file(logName,overwrite=False)
logging.info("Correction of automatic onsets/offsets detection:")

Load raw data

In [ ]:
raw = mne.io.read_raw_bdf(fname, preload=True, stim_channel='Status')

# Pre-process signal
(Should  be identical to detectEMG preprocessing)

EMG bipolar reference

In [ ]:
mne.set_bipolar_reference(raw,anode=['EXG1','EXG3'], cathode=['EXG2','EXG4'],ch_name=['EMG_L','EMG_R'],\
copy=False)

Set channels

In [ ]:
raw.pick(['EMG_L','EMG_R'])
raw.set_channel_types({'EMG_L':'emg',
                       'EMG_R':'emg'
                       })

emg_channels_idx = {0: 'EMG_L', 1: 'EMG_R'}
channel_names = list(emg_channels_idx.values())

Filter EMG and force signals

In [ ]:
raw = myo.use_mne.apply_filter(raw, ch_names = ['EMG_L','EMG_R'],low_cutoff = 10)
logging.info("\tHigh pass filtering of EMG traces at 10Hz")

Get pre-processed raw data

In [ ]:
data = raw.get_data(channel_names)

# Load automatic detection events and segment

Load events

In [ ]:
fname_mrk = os.path.join(path_mrk, nameSubj+'_detect_emg.csv') 
#fname_mrk = os.path.join(path_corrected_mrk, nameSubj+'_corrected_evts.csv') 
events = myo.load_continuous(fname_mrk,\
                             col_sample=0, col_code=2, col_chan=3,\
                             sf=raw.info['sfreq'])
logging.info("\tLoad events from " + fname_mrk)

Define segmentation window

In [ ]:
tmin = -.5
tmax = 1.5
code_t0 = list(stim_id.values())

# Start visualization application

 **!!! IMPORTANT !!!**
 
 Do not run the cell below twice, this would cause your kernel stop working. If you have closed viz window and want to see it again, just run the command viz.show() (next cell).

In [ ]:
viz = myo.Viz(sys.argv)
viz.load_data(data, events, code_t0,\
              tmin=tmin, tmax=tmax,\
              code_movable_1=emg_id['onset'], code_movable_2=emg_id['offset'],\
              sync_chan=[[0,1],[0,1],[2,3],[2,3]],random_order=False)

# Open viz window for correction

A complete description is available in vizEMG guide, here is a reminder of useful commands:
* Below signals, use the ‘Navig.’ tab to navigate across trials, and the ‘Config.’ tab to configurate data views and key shortcuts to jump to next/previous trial,
* Use left and right mouse clicks in data view windows or data axis to pan and zoom in and out signals,
* Vertical lines show marker events. Fixed events are displayed in yellow, no interaction is possible with those. Dark and light blue lines show events resulting from onset and offset automatic detection. With those, you can:
    * left click on the marker + drag to adjust event time position,
    * right click on the marker to suppress the event,
    * Ctrl key + left click to add an onset (dark blue) event,
    * Ctrkey + right click to add an offset (light blue) event.
* Markers modifications are automatically validated by jumping to a new trial.

In [ ]:
viz.show()

# Extract corrected events and save

In [ ]:
corrected_events = viz.get_events()

Check that we have the same number of onset and offset everywhere. To skip this step, uncomment the two next cells.

In [ ]:
corrected_epochs_events = corrected_events.segment(code_t0=code_t0)

In [ ]:
to_print = []

for t in range(corrected_epochs_events.nb_trials()):
    for c in range(data.shape[0]):
        
        tmin = corrected_epochs_events.tmin.time[t]
        if t < corrected_epochs_events.nb_trials()-1:
            tmax = corrected_epochs_events.tmin.time[t+1]
        else: tmax = data.shape[1] 
            
        onsets = corrected_events.find_and_get_events(code=emg_id['onset'], chan=c, tmin=tmin, tmax=tmax, print_find_evt=False)
        offsets = corrected_events.find_and_get_events(code=emg_id['offset'], chan=c, tmin=tmin, tmax=tmax,print_find_evt=False)
        
        if (onsets.nb_events() != offsets.nb_events()) or ((offsets.lat.time - onsets.lat.time) < 0).any():
#            to_print.append((('Trial {} on channel {} does not have same number of onset and offset, please correct!').format(t,c)))
            to_print.append((('Trial {} on channel {} does not have same number of onset and offset: onsets at {} offsets at {}')\
                              .format(t,c,onsets.lat.time-corrected_epochs_events.tmin.time[t],offsets.lat.time-corrected_epochs_events.tmin.time[t])))

if len(to_print) > 0:
    print(to_print)
    viz.show()

# Do not forget to SAVE corrected events !

In [ ]:
corrected_events = viz.get_events()
corrected_events.to_csv(os.path.join(path_corrected_mrk,nameSubj +'_corrected_evts.csv'),\
                        header="sample,time,code,chan",\
                        sep=',', save_sample=True, save_time=True, save_code=True, save_chan=True,\
                        save_trial_idx=False)